In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 비축 물량 신뢰도 추정을 위한 계층적 베이즈 분석\n",
    "\n",
    "## 1. 프로젝트 개요\n",
    "\n",
    "본 프로젝트는 제한된 수량의 샘플 테스트 데이터만을 사용하여, 전체 비축 물량(LOT)의 현재 신뢰도를 통계적으로 추정하는 것을 목표로 합니다.\n",
    "\n",
    "특히, 미래에 대한 불확실한 가정을 **'낙관적 시나리오'**와 **'보수적 시나리오'**로 나누어 모델링하고, 각 시나리오가 전체 비축 물량의 신뢰도 추정치에 어떤 영향을 미치는지 비교 분석합니다.\n",
    "\n",
    "## 2. 분석 단계\n",
    "1.  **환경 설정**: 분석에 필요한 라이브러리를 불러오고 기본 설정을 수행합니다.\n",
    "2.  **데이터 준비**: 가상의 시험 데이터와 전체 LOT 목록을 생성합니다.\n",
    "3.  **베이즈 모델 정의**: PyMC를 사용하여 계층적 베이즈 모델을 함수로 정의합니다.\n",
    "4.  **시나리오 실행**: '낙관적' 및 '보수적' 시나리오에 따라 모델을 실행하고 결과를 저장합니다.\n",
    "5.  **결과 분석 및 시각화**: \n",
    "    - 전체 평균 신뢰도 비교\n",
    "    - 모델의 주요 파라미터 비교\n",
    "    - 개별 LOT 신뢰도 비교"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 환경 설정 및 라이브러리 임포트"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pytensor\n",
    "pytensor.config.cxx = \"\"\n",
    "\n",
    "import pymc as pm\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import arviz as az\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from typing import Dict, Any, Tuple, List\n",
    "\n",
    "# Matplotlib 시각화 설정\n",
    "%matplotlib inline\n",
    "try:\n",
    "    plt.rcParams['font.family'] = 'Malgun Gothic' # Windows\n",
    "except:\n",
    "    plt.rcParams['font.family'] = 'AppleGothic' # Mac\n",
    "plt.rcParams['axes.unicode_minus'] = False\n",
    "sns.set_theme(style=\"whitegrid\", font=plt.rcParams['font.family'])\n",
    "\n",
    "# 상수 정의\n",
    "CURRENT_YEAR = 2025\n",
    "SCENARIO_COLORS = {\n",
    "    \"낙관적 (Optimistic)\": \"cornflowerblue\",\n",
    "    \"보수적 (Pessimistic)\": \"salmon\"\n",
    "}"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 데이터 준비\n",
    "분석에 사용할 시험 데이터와 전체 LOT 목록 및 인덱스를 생성합니다. 여기서는 2015년에 생산된 3개 LOT에 대한 시험 결과만 있다고 가정합니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def prepare_data_and_indices() -> Tuple[pd.DataFrame, Dict[str, Any]]:\n",
    "    \"\"\"분석에 필요한 데이터와 인덱스를 준비합니다.\"\"\"\n",
    "    data = pd.DataFrame({\n",
    "        'production_lot': ['2015-LOT-01', '2015-LOT-02', '2015-LOT-04'],\n",
    "        'num_tested': [11, 10, 12],\n",
    "        'num_failures': [0, 1, 1]\n",
    "    })\n",
    "    data['num_success'] = data['num_tested'] - data['num_failures']\n",
    "\n",
    "    all_years = np.arange(2015, 2020)\n",
    "    all_lots = [f\"{year}-LOT-{i:02d}\" for year in all_years for i in range(1, 7)]\n",
    "    lot_map = {lot: i for i, lot in enumerate(all_lots)}\n",
    "    year_map = {year: i for i, year in enumerate(all_years)}\n",
    "    year_of_lot = np.array([int(lot.split('-')[0]) for lot in all_lots])\n",
    "    year_idx_of_lot = np.array([year_map[y] for y in year_of_lot])\n",
    "    observed_lot_idx = [lot_map[lot] for lot in data['production_lot']]\n",
    "    \n",
    "    indices = {\n",
    "        \"all_lots\": all_lots,\n",
    "        \"all_years\": all_years,\n",
    "        \"year_idx_of_lot\": year_idx_of_lot,\n",
    "        \"observed_lot_idx\": observed_lot_idx,\n",
    "        \"year_of_lot\": year_of_lot\n",
    "    }\n",
    "    return data, indices\n",
    "\n",
    "# 데이터와 인덱스 생성\n",
    "data, indices = prepare_data_and_indices()\n",
    "\n",
    "print(\"--- 관측 데이터 ---\")\n",
    "print(data)\n",
    "print(f\"\\n--- 전체 LOT 개수: {len(indices['all_lots'])}개 ---\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 계층적 베이즈 모델 정의\n",
    "시나리오별 가정을 파라미터로 받아 동적으로 모델 구조를 변경하는 함수를 정의합니다.\n",
    "\n",
    "- **`degradation_effect_on_variance`**: `True`일 경우, 제품 연식이 오래될수록 신뢰도의 불확실성(`sigma_lot_effective`)이 증가하는 항을 모델에 추가합니다. (보수적 시나리오)\n",
    "- **`inter_year_sigma`, `intra_lot_sigma`**: 각각 연도 간, LOT 간 변동성에 대한 사전확률의 강도를 조절합니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def run_bayesian_model(data: pd.DataFrame, indices: Dict[str, Any], model_params: Dict[str, Any]) -> az.InferenceData:\n",
    "    \"\"\"주어진 파라미터로 베이지안 모델을 실행하고 추론 결과를 반환합니다.\"\"\"\n",
    "    with pm.Model() as model:\n",
    "        # 최상위 계층: 전체 평균 신뢰도 (로짓 스케일)\n",
    "        mu_global_logit = pm.Normal('mu_global_logit', mu=3.89, sigma=0.5) # 약 98% 신뢰도에 대한 사전 믿음\n",
    "        \n",
    "        # 중간 계층: 연도별 변동성\n",
    "        sigma_year = pm.HalfNormal('sigma_year', sigma=model_params[\"inter_year_sigma\"])\n",
    "        \n",
    "        # 최하위 계층: LOT별 변동성 (기본값)\n",
    "        sigma_lot_base = pm.HalfNormal('sigma_lot_base', sigma=model_params[\"intra_lot_sigma\"])\n",
    "\n",
    "        # 시나리오 분기: 경년열화 효과를 분산에 반영할지 여부\n",
    "        if model_params[\"degradation_effect_on_variance\"]:\n",
    "            variance_degradation_rate = pm.HalfNormal('variance_degradation_rate', sigma=0.05)\n",
    "            age_of_lot = CURRENT_YEAR - indices[\"year_of_lot\"]\n",
    "            sigma_lot_effective = pm.Deterministic('sigma_lot_effective', sigma_lot_base + age_of_lot * variance_degradation_rate)\n",
    "        else:\n",
    "            sigma_lot_effective = pm.Deterministic('sigma_lot_effective', sigma_lot_base)\n",
    "\n",
    "        # 계층 구조 정의\n",
    "        theta_year = pm.Normal('theta_year', mu=mu_global_logit, sigma=sigma_year, shape=len(indices[\"all_years\"]))\n",
    "        theta_lot = pm.Normal('theta_lot', mu=theta_year[indices[\"year_idx_of_lot\"]], sigma=sigma_lot_effective, shape=len(indices[\"all_lots\"]))\n",
    "        reliability_lot = pm.Deterministic('reliability_lot', pm.invlogit(theta_lot))\n",
    "\n",
    "        # 가능도: 관측 데이터와 모델 연결\n",
    "        y_obs = pm.Binomial('y_obs', n=data['num_tested'].values, p=reliability_lot[indices[\"observed_lot_idx\"]], observed=data['num_success'].values)\n",
    "        \n",
    "        # MCMC 샘플링\n",
    "        trace = pm.sample(2000, tune=1500, cores=1, return_inferencedata=True, random_seed=2024, progressbar=True)\n",
    "    return trace"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 시나리오별 모델 실행\n",
    "정의된 '낙관적' 및 '보수적' 시나리오에 따라 모델을 실행합니다. 이 과정은 컴퓨터 성능에 따라 수 분이 소요될 수 있습니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "scenarios = {\n",
    "    \"낙관적 (Optimistic)\": {\"inter_year_sigma\": 0.01, \"intra_lot_sigma\": 0.02, \"degradation_effect_on_variance\": False},\n",
    "    \"보수적 (Pessimistic)\": {\"inter_year_sigma\": 0.2, \"intra_lot_sigma\": 0.1, \"degradation_effect_on_variance\": True}\n",
    "}\n",
    "traces = {}\n",
    "\n",
    "for name, params in scenarios.items():\n",
    "    print(f\"\\n--- Running analysis for scenario: [{name}] ---\")\n",
    "    traces[name] = run_bayesian_model(data, indices, params)\n",
    "    print(f\"--- Scenario analysis complete: [{name}] ---\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 결과 분석 및 시각화\n",
    "두 시나리오의 실행 결과를 세 가지 다른 관점에서 비교하여 시각화합니다."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 5.1. 전체 평균 신뢰도 분포 비교\n",
    "각 시나리오에서 추정한 전체 LOT의 평균 신뢰도 분포를 비교합니다. 이를 통해 어떤 가정이 더 비관적인 결과를 도출하는지 한눈에 파악할 수 있습니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_stockpile_reliability_comparison(traces: Dict[str, az.InferenceData]):\n",
    "    \"\"\"여러 시나리오의 전체 비축물량 평균 신뢰도 분포를 비교하는 밀도 그림을 생성합니다.\"\"\"\n",
    "    plt.figure(figsize=(12, 8))\n",
    "    \n",
    "    print(\"\\n--- Overall Stockpile Reliability Analysis ---\")\n",
    "    for name, trace in traces.items():\n",
    "        posterior_samples = trace.posterior['reliability_lot'].values\n",
    "        mean_per_draw = posterior_samples.mean(axis=-1).flatten()\n",
    "        \n",
    "        sns.kdeplot(\n",
    "            mean_per_draw, \n",
    "            label=f'시나리오: {name}', \n",
    "            fill=True, \n",
    "            alpha=0.6, \n",
    "            color=SCENARIO_COLORS.get(name)\n",
    "        )\n",
    "        \n",
    "        hdi_bounds = az.hdi(mean_per_draw, hdi_prob=0.9)\n",
    "        plt.axvspan(hdi_bounds[0], hdi_bounds[1], color=SCENARIO_COLORS.get(name), alpha=0.1)\n",
    "        print(f\"[{name} 시나리오] 90% 신뢰구간(HDI): [{hdi_bounds[0]:.4f}, {hdi_bounds[1]:.4f}]\")\n",
    "    \n",
    "    plt.title('시나리오별 전체 비축물량 평균 신뢰도 분포 비교', fontsize=20, pad=20)\n",
    "    plt.axvline(x=0.98, color='grey', linestyle='--', linewidth=1.5, label='목표 신뢰도 (98%)')\n",
    "    plt.text(0.98, plt.ylim()[1]*0.9, ' 목표 신뢰도', color='dimgray', fontsize=12, ha='left')\n",
    "    plt.xlabel('전체 평균 신뢰도', fontsize=15)\n",
    "    plt.ylabel('확률 밀도', fontsize=15)\n",
    "    plt.legend(title='시나리오', fontsize=13, title_fontsize=14)\n",
    "    plt.tick_params(axis='both', which='major', labelsize=12)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "# 플롯 생성\n",
    "plot_stockpile_reliability_comparison(traces)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 5.2. 주요 파라미터 사후 분포 비교\n",
    "두 시나리오의 결과가 왜 달라졌는지, 그 원인을 파악하기 위해 모델의 핵심 파라미터(`sigma_year`, `sigma_lot_base` 등)가 어떻게 추정되었는지 비교합니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_posterior_distributions(traces: Dict[str, az.InferenceData]):\n",
    "    \"\"\"모델의 주요 파라미터에 대한 사후 분포를 시각화합니다.\"\"\"\n",
    "    all_params = ['mu_global_logit', 'sigma_year', 'sigma_lot_base', 'variance_degradation_rate']\n",
    "    \n",
    "    for param in all_params:\n",
    "        plt.figure(figsize=(10, 6))\n",
    "        has_data = False\n",
    "        \n",
    "        for name, trace in traces.items():\n",
    "            if param in trace.posterior:\n",
    "                has_data = True\n",
    "                samples = trace.posterior[param].values.flatten()\n",
    "                sns.kdeplot(\n",
    "                    samples,\n",
    "                    label=name,\n",
    "                    color=SCENARIO_COLORS.get(name),\n",
    "                    fill=True,\n",
    "                    alpha=0.6\n",
    "                )\n",
    "        \n",
    "        if not has_data:\n",
    "            plt.close() # 데이터가 없는 파라미터(예: 낙관적 시나리오의 variance_degradation_rate)는 플롯을 생성하지 않음\n",
    "            continue\n",
    "\n",
    "        plt.title(f'주요 파라미터 사후 분포 비교: {param}', fontsize=20, pad=20)\n",
    "        plt.xlabel('파라미터 값', fontsize=15)\n",
    "        plt.ylabel('확률 밀도', fontsize=15)\n",
    "        plt.legend(title='시나리오', fontsize=13, title_fontsize=14)\n",
    "        plt.tick_params(axis='both', which='major', labelsize=12)\n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "\n",
    "# 플롯 생성\n",
    "plot_posterior_distributions(traces)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 5.3. 개별 LOT 신뢰도 비교 (Forest Plot)\n",
    "가장 상세한 수준의 분석으로, 각 LOT별 신뢰도 추정치(평균과 90% 신뢰구간)를 Forest Plot으로 시각화합니다. 이를 통해 어떤 LOT의 신뢰도가 낮게 추정되는지, 불확실성은 얼마나 큰지 등을 개별적으로 확인할 수 있습니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_lot_reliability_forest(traces: Dict[str, az.InferenceData], indices: Dict[str, Any]):\n",
    "    \"\"\"시나리오별로 각 LOT의 신뢰도를 Forest Plot으로 시각화합니다.\"\"\"\n",
    "    for name, trace in traces.items():\n",
    "        plt.figure(figsize=(16, 8))\n",
    "        \n",
    "        # 데이터 추출\n",
    "        posterior_data = trace.posterior['reliability_lot']\n",
    "        hdi_data = az.hdi(posterior_data, hdi_prob=0.9)['reliability_lot'].values\n",
    "        mean_data = posterior_data.mean(dim=['chain', 'draw']).values\n",
    "        \n",
    "        lot_indices = np.arange(len(indices[\"all_lots\"]))\n",
    "        \n",
    "        # 90% HDI 에러바\n",
    "        plt.errorbar(x=lot_indices, y=mean_data, yerr=[mean_data - hdi_data[:, 0], hdi_data[:, 1] - mean_data],\n",
    "                     fmt='none', # 점은 따로 그림\n",
    "                     color='lightgray', elinewidth=2, capsize=5, label='90% HDI (신뢰구간)')\n",
    "        \n",
    "        # 평균 추정치 점\n",
    "        plt.plot(lot_indices, mean_data, 'o', color=SCENARIO_COLORS.get(name), markersize=8, label='평균 추정치')\n",
    "        \n",
    "        plt.xticks(ticks=lot_indices, labels=indices[\"all_lots\"], rotation=90, fontsize=10)\n",
    "        plt.title(f'[{name} 시나리오] LOT별 신뢰도 추정', fontsize=20, pad=20)\n",
    "        plt.ylabel('추정 신뢰도', fontsize=15)\n",
    "        plt.xlabel('생산 LOT', fontsize=15)\n",
    "        plt.legend(fontsize=12)\n",
    "        plt.ylim(0.9, 1.0) # y축 범위를 조정하여 가독성 향상\n",
    "        plt.tight_layout()\n",
    "        plt.show()\n",
    "\n",
    "# 플롯 생성\n",
    "plot_lot_reliability_forest(traces, indices)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.9"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
